<a href="https://colab.research.google.com/github/chopra2408/finetuning/blob/main/finetuned_llama_2_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip -q install peft accelerate bitsandbytes transformers datasets

ERROR: unknown command "-"


In [2]:
#peft is parameter efficient finetuning.
# - needed to implement LoRA Low Rank Adaptation,
# - allows us to finetune large models
#accelerate we need for optimization and harware utilization
# - helps distribute training across gpu and implements out of like precision training
#bitsandbytes for 4bit quantization
#transformers main library for importing autotokenizer
#We gonna use sft supervise finetuning

In [19]:
!pip -q install gputil huggingface_hub

In [49]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
  print("GPU available")
else:
  print("Not available")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% | 56% |
GPU available


In [21]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from huggingface_hub import notebook_login
from datasets import load_dataset

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

In [22]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [23]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, #loading model in 4b
    bnb_4bit_use_double_quant=True, #double quantization in 4bit saves memory
    bnb_4bit_quant_type="nf4", #optimized for normal float 4 optimized for normal distribution for neural networks, preserving accuracy
    bnb_4bit_compute_dtype=torch.bfloat16 #datatype for finetuning
)

model = AutoModelForCausalLM.from_pretrained(
      base_model_id, quantization_config=bnb_config,
      device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
!git clone https://github.com/poloclub/fine-tuning-LLMs.git

fatal: destination path 'fine-tuning-LLMs' already exists and is not an empty directory.


In [25]:
train_dataset = load_dataset(
    "text", data_files={
        "train": [
            "/content/fine-tuning-LLMs/data/hawaii_wf_1.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_2.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_3.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_4.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_5.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_6.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_7.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_8.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_9.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_10.txt"
            ]
        }, split="train"
    )

In [26]:
train_dataset = load_dataset("text", data_files={"train":
                                                 [
            "/content/fine-tuning-LLMs/data/hawaii_wf_1.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_2.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_3.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_4.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_5.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_6.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_7.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_8.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_9.txt",
            "/content/fine-tuning-LLMs/data/hawaii_wf_10.txt"
            ]}, split="train")

In [27]:
train_dataset["text"][:10]

['POLICE DEPARTMENT',
 'COUNTY OF MAUI',
 '55 MAHALANI STREET',
 'WAILUKU, MAUI, HAWAII 96793',
 'TELEPHONE: (808) 244-6400',
 'FAX: (808) 244-6411',
 'JOHN PELLETIER',
 'CHIEF OF POLICE',
 'WADE M. MAEDA',
 'DEPUTY CHIEF OF POLICE']

In [28]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, user_fast=False, trust_remote_code=True, add_eos_token=True)
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [29]:
tokenized_train_dataset = []
for phrase in train_dataset:
  tokenized_train_dataset.append(tokenizer(phrase["text"]))

In [30]:
tokenized_train_dataset[:1]

[{'input_ids': [1, 349, 5607, 12107, 5012, 26092, 13780, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}]

In [31]:
tokenizer.eos_token

'</s>'

In [32]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8, #balance, lower r means less expressive, r=8 update matrices by rank 8
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [34]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir="./finetuned_model",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        #batch processing flow ie how we are processing entire dataset
        #iteration 1 => forward pass => gradient calculated and stored
        #iteration 2 => forward pass => gradient calculated and combined with previous and given to model object
        #remembers the gradient which is calculated, combines the gradient from specific formula and then update the model
        warmup_steps=2,
        num_train_epochs=10,
        learning_rate=1e-4,
        max_steps=1500,
        bf16=False,
        optim="paged_adamw_8bit",
        #optimizer paged adamw is a memory efficient optimizer
        #standard adamw means all the parameters are stored in memory
        #paged adamw means they are paged optimized states is 8 bit qunatized states are stored in memory
        logging_dir="./logs",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=10,
    ),
  data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache=False
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chopranishant71 (chopranishant71-kindlebit-solutions-pvt) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.911000
20,3.214200
30,2.915700
40,2.884300
50,2.761700
60,2.632800
70,2.813600
80,2.643600
90,2.402000
100,2.661400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=1500, training_loss=1.6649405212402344, metrics={'train_runtime': 9691.9748, 'train_samples_per_second': 0.619, 'train_steps_per_second': 0.155, 'total_flos': 5407704170373120.0, 'train_loss': 1.6649405212402344, 'epoch': 3.571768910065515})

In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer, LlamaForCausalLM
from peft import PeftModel

base_model_id = "meta-llama/Llama-2-7b-chat-hf"
nf4Config = BitsAndBytesConfig(
    load_in_4bit=True, #loading model in 4b
    bnb_4bit_use_double_quant=True, #double quantization in 4bit saves memory
    bnb_4bit_quant_type="nf4", #optimized for normal float 4 optimized for normal distribution for neural networks, preserving accuracy
    bnb_4bit_compute_dtype=torch.bfloat16 #datatype for finetuning
)

tokenizer = LlamaTokenizer.from_pretrained(
    base_model_id, user_fast=False,
    trust_remote_code=True,
    add_eos_token=True
  )

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [44]:
tokenizer = LlamaTokenizer.from_pretrained(
    base_model_id, user_fast=False,
    trust_remote_code=True,
    add_eos_token=True
  )

modelFinetuned = PeftModel.from_pretrained(base_model, "/finetuned_model/checkpoint-1500/")

In [55]:
user_question = "how did fire started"

eval_prompt = f"""
You are an AI assistant that provides accurate and concise answers.

Question: {user_question}

Answer the question directly and succinctly without additional explanation.

"""

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()


You are an AI assistant that provides accurate and concise answers.  

Question: how did fire started  

Answer the question directly and succinctly without additional explanation.

 obviously, this is not all of the information that is available. there are many more details that have been reported and/or discovered. however, this should give you a good idea of what is available. 

please keep in mind that this is not all inclusive and there may be more information out there. if you have any specific questions, please let me know and i will do my best to assist. 

in addition, please be aware that this information is subject to change and may not be entirely accurate or up to date. please do your own research and verify any information you come across. 

i hope this helps! if you have any other questions, please don't hesitate to ask.
